# Boston House Price Predictor using ML

Project Information:
This is my 1st Machine learning Project. I am taking data of boston house pricing from kaggle and apllying machine learning
to predict prices if we have provided with new feature of a house.

Date - 10/07/2022

Time - 2:56 AM

Author - Abhijit Kumar(Nit Durgapur CSE)

##Atrributes of the data

1. CRIM: per capita crime rate by town
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS: proportion of non-retail business acres per town
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX: nitric oxides concentration (parts per 10 million)
6. RM: average number of rooms per dwelling
7. AGE: proportion of owner-occupied units built prior to 1940
8. DIS: weighted distances to five Boston employment centres
9. RAD: index of accessibility to radial highways
10. TAX: full-value property-tax rate per $10,000
11. PTRATIO: pupil-teacher ratio by town
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT: % lower status of the population
14. MEDV: Median value of owner-occupied homes in

In [ ]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split #used for spliting dataset into training and testing
from sklearn import metrics
from sklearn.model_selection import cross_val_score
#import sklearn.preprocessing, sklearn.decomposition, \sklearn.linear_model, sklearn.pipeline, sklearn.metrics, \sklearn.compose
# import sklearn_pandas


In [ ]:
feature_names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]
#reading data in csv file and creating a dataframe
csv_df=pd.read_csv("/Boston_house_price_data.csv",names=feature_names)
csv_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


##Analyzing the Data

In [ ]:
# Steps
# 1) csv_df.info()
# 2) csv_df['column_name'].value_counts()
# 3) csv_df.describe()
# 4) csv_df.hist(bins=50,figsize=(20,10))
## Here we also seperate our data into features and labels to simplify the process of splitting
labels_data=csv_df['MEDV'].copy(deep=True)
features_data=csv_df.drop('MEDV',axis=1).copy(deep=True)

##Spliting Training and testing data
At this point we should  split our data into training and testing so that we will not conclude unnecessary patterns from all data.

*Proablem to be taken care of:-*

But we have to take take of uniformness of data while spliting Eg:- if a feature has binary values and 100 times it exists as 1 and 20 times it exists as 0 .So there might be a case that we accidently split the data in such a way that all ones came in training so our model will make wrong assumtion (our model think that there is only one value possible for this feature.)

Solution ➡

We use stratified sampling.for that we use inbuilt class of sklearn that is train_test_split(We can also use StratifiedShuffleSplit
`(sklearn.model_selection.train_test_split)`

##inside this train_test_split method there is option for stratified spliting.
`sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)`

In [ ]:
#here we done stratified sampling on the basis of 'CHAS' feature
features,features_test,labels,labels_test=train_test_split(features_data,labels_data,test_size=0.2,train_size=0.8,random_state=42,shuffle=True,stratify=features_data['CHAS'])
X_test=features_test.to_numpy()
Y_test=labels_test.to_numpy()
#From here on we do not touch our test datas we only work on our trianing data

### Exploring Correlation
Now we try to find or explore the correlation of every feature with label for that we use df.corrwith() function .it output float value between -1 to 1 where -1 -> very strong negetive relation and +1 -> very strong positve relation.
We can also use seaborn heatmap plotting or scatter_matrix plotting using pandas.plotting



In [ ]:
# We also created a fulldata_train by adding features and labels for future operations
fulldata_train=features.join(labels).copy(deep=True)
#finding and plottinf correlation matrix
corr_matrix=fulldata_train.corr()
#heatmap plotting using seaborn
'''
import seaborn as sns
sns.heatmap(corr_matrix.loc[:,'MEDV':])
'''
#scatter_matrix plotting using pandas.plotting
'''
from pandas.plotting import scatter_matrix
#its better that you use attributes and see plots and then change items in it and again see plots its better otherwise we can see heatmap
attributes=['CRIM','RM','AGE','MEDV']
scatter_matrix(fulldata_train[attributes],alpha=0.8,figsize=(12,10))
plt.show()
'''

"\nfrom pandas.plotting import scatter_matrix\n#its better that you use attributes and see plots and then change items in it and again see plots its better otherwise we can see heatmap\nattributes=['CRIM','RM','AGE','MEDV']\nscatter_matrix(fulldata_train[attributes],alpha=0.8,figsize=(12,10))\nplt.show()\n"

##Taking care of missing attributes
you have three option to deal with it

1) Get rid of the missing data points

`df.dropna(subset=["attribute_name"])` #it does not change original df so for changing in original df we have to use inplace=True

2)Get rid of whole attribute

`df.drop("attribute_name",axis=1,inplace=True)`

3)reaplace NA value with some other value (that do not chaange in your model results)

(we are replacing here with median)

`median=df["atrribute_name"].median()`

`df["attribute_name"].fillna(median)`

##But for doing this task we already have an inbuilt method (Imputer)
`from sklearn.impute import SimpleImputer`

`imputer=SimpleImputer(strategy="median")`

`imputer.fit(df)`

`imputed_array=imputer.transform(df)` #this is an ndarray so we can convert it into pandas dataframe

`tranformed_df=pd.DataFrame(imputed_array,columns=df.columns)` #tranformed_df stores df with all NA replaced by their respective medians`

we can also do above steps by making a pipeline (recommended).


In [1]:
'''
# we can do this in pipeline
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
imputer.fit(fulldata_train)
imputed_array=imputer.transform(fulldata_train)
tranformed_df=pd.DataFrame(imputed_array,columns=fulldata_train.columns)
'''

'\n# we can do this in pipeline\nfrom sklearn.impute import SimpleImputer\nimputer=SimpleImputer(strategy="median")\nimputer.fit(fulldata_train)\nimputed_array=imputer.transform(fulldata_train)\ntranformed_df=pd.DataFrame(imputed_array,columns=fulldata_train.columns)\n'

##Scikit-learn Design
Primary three type of objects:-

**1) Estimators**

It estimates some parameters based on the datasets ex:- Imputer

It has a fit method-> fits the dataset and calculate internal parameters and tranform method.

**2) Tranformers**

transform method takes input and return output based on the learning from fit() of estimiator.

It also as a convineance function called `fit_transform()` {faster} which fits and then transform simultaneously.

**3) Predictors**

LinearRegression model is an Exaple of predictor.It has two common functions a)`fit()` -> fits the training data and create model b)`predict()`-> predicts the output based on the learning of the fit.

it also gives the `score()` function which will evaluate the predictions.

## Features-scaling
**Why we need this concept?**

Value of different features can be very diffrent ex:-faetureA values can range in [0,1] and value of featureB can range in [-1000,1000] so for better modelling we somehow try that if we can scale all feature in same range that's why we use feature scaling.

**Primary there are two types of feature scaling.**

1) Min-Max Scaling (Normalisation)

Ex:- let there be a featureA which minmum value is minm and maximum value is maxm.so for each value val of featureA  we do

scaled_val=(val-min)/(max-min) (#range of scaled_val is [0,1])

sklearn provides a class called `MinMaxScaler` for minmax scaling.

2) Standardization

val->values of feature, mean-> mean of all values of feature , std-> standard deviation of feature.

scaled_value=(val-mean)/(std)

sklearn provides a class called `StandardScaler` for this scaling.



In [ ]:
'''
# we can do this in pipeline

from sklearn.preprocessing import StandardScaler
standard_scaler=StandardScaler()
features=pd.DataFrame(standard_scaler.fit_transform(features),column=features.columns)
features.head()
'''

'\n# we can do this in pipeline\n\nfrom sklearn.preprocessing import StandardScaler\nstandard_scaler=StandardScaler()\nfeatures=pd.DataFrame(standard_scaler.fit_transform(features),column=features.columns)\nfeatures.head()\n'

#Creating a Pipeline
In a pipeline we add diffrent processes.a pipeline takes the data and run the process on it and pass the output to the next process to execute.

`from sklearn.pipeline import Pipeline`

`from sklearn.preprocessing import StandardScaler`

`from sklearn.model_selection import SimpleImputer`

`my_pipeline=Pipeline([`

  `('imputer',SimpleImputer(strategy="median")),#We can add as many process as we want`

  `('std_scaler',StandardScaler())`

`])`

After that we can directly use fit_tranform method of pipeline.and this fit transform return a numpy array.

In [ ]:
# We will crete a pipeline here and add Imputer and scaling into pipeline
my_pipeline=Pipeline([
('imputer',SimpleImputer(strategy="median")),#We can add as many process as we want
('std_scaler',StandardScaler())
])
'''
# Parllel plotting of dataframe
fulldata_train=pd.DataFrame(my_pipeline.fit_transform(fulldata_train),columns=fulldata_train.columns)
ax=pd.plotting.parallel_coordinates(fulldata_train,'MEDV')
legend =ax.legend()
legend.remove()
plt.show()
'''
# fit and transform the pipeline on features
#we are just giving a new name to the features as X_train and labels to Y_train
X_train=my_pipeline.fit_transform(features)
Y_train=labels.to_numpy()

# this below step is very important because if our training feature have passed through pipeline then our testing features should also
# pass through pipeline
X_test=my_pipeline.fit_transform(X_test)

#Select your desired model for modelling
Example models for Regression

**DecisionTreeRegressor**

syntax

`from sklearn.tree import DecisionTreeRegressor`

**Gradient Boosting Regression**

Syntax

`from sklearn.ensemble import GradientBoostingRegressor`


**Elastic Net Regression**

Syntax

`from sklearn.linear_model import ElasticNe`t

**Stochastic Gradient Descent Regression**

Syntax

`from sklearn.linear_model import SGDRegressor`

**Support Vector Machine**

Syntax

`from sklearn.svm import SVR`

**Bayesian Ridge Regression**

Syntax

`from sklearn.linear_model import BayesianRidge`

**CatBoost Regressor**

Syntax

`from catboost import CatBoostRegressor`

**Kernel Ridge Regression**

Syntax

`from sklearn.kernel_ridge import KernelRidge`

**Linear Regression**

Syntax

`from sklearn.linear_model import LinearRegression`

**KNeighbors Regressor**

Syntax

`from sklearn.neighbors import KNeighborsRegressor`

**RandomForestRegressor**

Syntax

`from sklearn.ensemble import RandomForestRegressor`

**XGBoost Regressor**

Syntax

`from xgboost.sklearn import XGBRegressor`

**LGBM Regressor**

Syntax

`from lightgbm import LGBMRegressor`

**You can allways check for k-fold cross validation (to check if our model get overfitted on training dataset)**

K-fold cross validation makes  k groups randomly in our dataset and run our model on it and calculate the error for every fold and return it.

Syntax

`from sklearn.model_selection import cross_val_score`

`cross_val_score(model/estimator,x,y,scoring="neg_mean_squared_error",cv=10(->groups of folding))`

it return neg of mean sqaured error so we have to make it positive and then take its root to get root mean squred  error.


In [ ]:
# import your model and start modelling
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import SGDRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor()
model.fit(X_train,Y_train)
# model.score(features,labels)
# model.coef_
# model.intercept_
Y_predict=model.predict(X_test)

# calulating root mean squared error
rmse=np.sqrt(metrics.mean_squared_error(Y_test,Y_predict))
print("Rmse = ",rmse)

#Calculating K-fold cross validation (to check if our model got overfitted)
rmse_cross_val_scores=np.sqrt(-1*cross_val_score(model,X_train,Y_train,scoring="neg_mean_squared_error",cv=10))
print("Rmse cross val scores = \n",rmse_cross_val_scores)

Rmse =  3.3789639901985953
Rmse cross val scores = 
 [2.87667815 2.90645052 4.44626112 2.62887072 3.43234611 2.54121222
 4.512502   3.27714691 3.44440933 3.21893902]


#Results:-
**for LinearRegression**

Rmse =  4.22823975454173

Rmse cross val scores = 
 [4.21674442 4.26026816 5.1071608  3.82881892 5.34093789 4.3785611
 7.47384779 5.48226252 4.14885722 6.0669122 ]

**for DecisionTreeRegressor**

Rmse =  4.579825752315106

Rmse cross val scores =
 [4.00051826 3.99264568 5.5571751  4.12458424 4.09615674 2.79986607
 5.19155564 3.746465   3.25326759 4.28893926]

 **for SGDRegressor**
 
 Rmse =  4.252223739738116
 
Rmse cross val scores = 
 [4.24276292 4.20132664 5.16754377 3.81191694 5.30153767 4.41060904
 7.56190827 5.47565146 4.16871895 6.12701213]

 **for KNeighborsRegressor**
 
 Rmse =  4.312169282573288
 
Rmse cross val scores = 
 [3.61967119 4.70763158 5.16523463 4.45849805 4.6198777  3.46735923
 8.47209655 5.52478235 3.22961608 4.1353174 ]

**for RandomForestRegressor**

Rmse =  3.3789639901985953

Rmse cross val scores = 
 [2.87667815 2.90645052 4.44626112 2.62887072 3.43234611 2.54121222
 4.512502   3.27714691 3.44440933 3.21893902]

 **Here we can clearly see that our *RandomForestRegressor* model Wins here(as it is giving lowest rmse)**  

 🙂🙂

 (Any edits/suggestions are most welcome) 🙏

***©Abhijit Kumar (NIT Duragapur CSE)***